In [1]:
import pandas as pd
import numpy as np
#import polars as pl
import ast

In [2]:
#df_movies = pl.read_csv("Dataset/movies_dataset.csv",ignore_errors=True)
df_movies = pd.read_parquet("dataset/movies_dataset.parquet")

In [3]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45460 non-null  float64
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

- ## Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

**Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.**

In [4]:
print(df_movies['spoken_languages'][0])
print(df_movies['production_companies'][0])
print(df_movies['production_countries'][0])
print(df_movies['genres'][0])
print(df_movies['belongs_to_collection'][0])

[{'iso_639_1': 'en', 'name': 'English'}]
[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}


In [5]:
#df_filtrado = df_movies.clone()
df_filtrado = df_movies.copy()
df_filtrado = df_filtrado[['belongs_to_collection','genres', 'production_companies','production_countries','spoken_languages']]
df_movies = df_movies.drop(['belongs_to_collection','genres', 'production_companies','production_countries','spoken_languages'], axis=1)

In [6]:
df_filtrado['genres'] = df_filtrado['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
#df_filtrado['belongs_to_collection'] = df_filtrado['belongs_to_collection'].apply(ast.literal_eval)
df_filtrado['belongs_to_collection'] = df_filtrado['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else None)
df_filtrado['production_companies'] = df_filtrado['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
df_filtrado['production_countries'] = df_filtrado['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
df_filtrado['spoken_languages'] = df_filtrado['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)

In [7]:
df_filtrado['id_genres'] = df_filtrado['genres'].apply(lambda x: [d['id'] for d in x])
df_filtrado['name_genres'] = df_filtrado['genres'].apply(lambda x: [d['name'] for d in x])

#df_filtrado['id_collection'] = df_filtrado['belongs_to_collection'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
#df_filtrado['name_collection'] = df_filtrado['belongs_to_collection'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])
#df_filtrado['poster_collection'] = df_filtrado['belongs_to_collection'].apply(lambda x: [d['poster_collection'] for d in x] if isinstance(x, list) else [])
#df_filtrado['backdrop_collection'] = df_filtrado['belongs_to_collection'].apply(lambda x: [d['backdrop_collection'] for d in x] if isinstance(x, list) else [])
df_filtrado['collection_id'] = df_filtrado['belongs_to_collection'].apply(lambda x: x['id'] if isinstance(x,dict) else None)
df_filtrado['collection_name'] = df_filtrado['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x,dict) else "No es una saga")
df_filtrado['collection_poster'] = df_filtrado['belongs_to_collection'].apply(lambda x: x['poster_path'] if isinstance(x,dict) else None)
df_filtrado['collection_backdrop'] = df_filtrado['belongs_to_collection'].apply(lambda x: x['backdrop_path'] if isinstance(x,dict) else None)


df_filtrado['id_prod_companies'] = df_filtrado['production_companies'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
#el codigo de abajo me lanza error: TypeError: 'float' object is not iterable
#df_filtrado['id_prod_companies'] = df_filtrado['production_companies'].apply(lambda x: [d['id'] for d in x] if x is not None else [])
#el codigo de abajo me lanza error: TypeError: 'NoneType' object is not iterable
#df_filtrado['id_prod_companies'] = df_filtrado['production_companies'].apply(lambda x: [d['id'] for d in x])
df_filtrado['name_prod_companies'] = df_filtrado['production_companies'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])
#df_filtrado['name_prod_companies'] = df_filtrado['production_companies'].apply(lambda x: [d['name'] for d in x])

#df_filtrado['iso_prod_countries'] = df_filtrado['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x])
df_filtrado['iso_prod_countries'] = df_filtrado['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x] if isinstance(x, list) else [])
df_filtrado['name_prod_countries'] = df_filtrado['production_countries'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

#df_filtrado['iso_spoken_languages'] = df_filtrado['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x])
df_filtrado['iso_spoken_languages'] = df_filtrado['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x] if isinstance(x, list) else [])
df_filtrado['name_spoken_languages'] = df_filtrado['spoken_languages'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [8]:
df_filtrado = df_filtrado.drop(['belongs_to_collection','genres','production_countries','spoken_languages','production_companies'], axis=1)

In [9]:
df_filtrado.head()

,id_genres,name_genres,collection_id,collection_name,collection_poster,collection_backdrop,id_prod_companies,name_prod_companies,iso_prod_countries,name_prod_countries,iso_spoken_languages,name_spoken_languages
0,"[16, 35, 10751]","[Animation, Comedy, Family]",10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,[3],[Pixar Animation Studios],[US],[United States of America],[en],[English]
1,"[12, 14, 10751]","[Adventure, Fantasy, Family]",NaN,No es una saga,None,None,"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]"
2,"[10749, 35]","[Romance, Comedy]",119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English]
3,"[35, 18, 10749]","[Comedy, Drama, Romance]",NaN,No es una saga,None,None,[306],[Twentieth Century Fox Film Corporation],[US],[United States of America],[en],[English]
4,[35],[Comedy],96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",[US],[United States of America],[en],[English]


In [10]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_genres              45466 non-null  object 
 1   name_genres            45466 non-null  object 
 2   collection_id          4491 non-null   float64
 3   collection_name        45466 non-null  object 
 4   collection_poster      3948 non-null   object 
 5   collection_backdrop    3263 non-null   object 
 6   id_prod_companies      45466 non-null  object 
 7   name_prod_companies    45466 non-null  object 
 8   iso_prod_countries     45466 non-null  object 
 9   name_prod_countries    45466 non-null  object 
 10  iso_spoken_languages   45466 non-null  object 
 11  name_spoken_languages  45466 non-null  object 
dtypes: float64(1), object(11)
memory usage: 4.2+ MB


In [12]:
df_filtrado.to_parquet("dataset_transformado/datos_secundarios_movies.parquet")

In [79]:
df_movies.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,None,Toy Story,False,7.7,5415.0
1,False,65000000,None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,0,None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,16000000,None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,0,None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


---
---

- ## Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.



In [80]:
print(df_movies['budget'].isnull().sum())
print(df_movies['revenue'].isnull().sum())
df_movies['revenue'].value_counts()

0
6


revenue
0.0           38052
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
              ...  
100000.0          1
15073942.0        1
2807854.0         1
1942423.0         1
23833131.0        1
Name: count, Length: 6863, dtype: int64

In [81]:
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['revenue'].isnull().sum()

np.int64(0)

In [82]:
df_movies['revenue'].value_counts()

revenue
0.0           38058
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
              ...  
100000.0          1
15073942.0        1
2807854.0         1
1942423.0         1
23833131.0        1
Name: count, Length: 6863, dtype: int64

---
---

- ## Los valores nulos del campo release_date deben eliminarse.



In [83]:
print(df_movies["release_date"].isnull().sum())

87


In [84]:
df_movies = df_movies.dropna(subset=['release_date'])
print(df_movies["release_date"].isnull().sum())

0


In [85]:
df_movies = df_movies.reset_index(drop=True)

---
---

- ## De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.



In [86]:
df_movies["fechas_validas"] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce')
fechas_invalidas = df_movies[df_movies["fechas_validas"].isna()]

In [87]:
fechas_invalidas

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
19714,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,1,0.0,NaN,None,None,None,None,NaN,NaN,NaT
29472,Rune Balot goes to a casino connected to the ...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,12,0.0,NaN,None,None,None,None,NaN,NaN,NaT
35543,Avalanche Sharks tells the story of a bikini ...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Avalanche Sharks,22,0.0,NaN,None,None,None,None,NaN,NaN,NaT


In [88]:
df_movies[19713:19716]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
19713,False,0,None,27058,tt0171227,en,Diplomatic Siege,When Steve Mitchell is clandestinely sent to t...,0.571035,/tLsQcikwkWKQiD7IHO0WIVe7m0Y.jpg,1999-11-17,0.0,90.0,Released,None,Diplomatic Siege,False,4.7,3.0,1999-11-17
19714,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,1,0.0,NaN,None,None,None,None,NaN,NaN,NaT
19715,False,21000000,http://silverliningsplaybookmovie.com/,82693,tt1045658,en,Silver Linings Playbook,After spending eight months in a mental instit...,14.488111,/ilrZAV2klTB0FLxLb01bOp5pzD9.jpg,2012-09-08,205738714.0,122.0,Released,Watch For The Signs,Silver Linings Playbook,False,7.0,4840.0,2012-09-08


In [89]:
df_movies[29471:29474]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
29471,False,0,http://m-scramble.jp/compression/,73529,tt1754177,ja,マルドゥック・スクランブル 圧縮,Rune Ballot is a down-and-out teen prostitute ...,2.640175,/A6UHcJ6S5f69DDWkiK10DTE6q8T.jpg,2010-11-06,0.0,69.0,Released,None,Mardock Scramble: The First Compression,False,7.1,23.0,2010-11-06
29472,Rune Balot goes to a casino connected to the ...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,12,0.0,NaN,None,None,None,None,NaN,NaN,NaT
29473,False,0,None,50794,tt0327698,en,The Even Stevens Movie,The Stevens think that they've won an all-expe...,3.352237,/iXq9a9QDq5ZNPDsHgM6HJgT42QM.jpg,2003-06-13,0.0,120.0,Released,None,The Even Stevens Movie,False,5.6,46.0,2003-06-13


In [90]:
df_movies[35542:35545]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
35542,False,0,None,89145,tt0040097,en,Angels' Alley,Slip invites his cousin Jimmy to stay with his...,0.14460,/2gsYyWFQITpOyI99qpJzG4GCImQ.jpg,1948-03-07,0.0,0.0,Released,TWO-FISTED DRAMA!,Angels' Alley,False,3.0,1.0,1948-03-07
35543,Avalanche Sharks tells the story of a bikini ...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Avalanche Sharks,22,0.0,NaN,None,None,None,None,NaN,NaN,NaT
35544,False,0,None,151911,tt0029924,en,Blockade,A simple peasant is forced to take up arms to ...,0.69411,/cYolP5lEMCCMpXMTAbuf07Il3sT.jpg,1938-06-17,0.0,85.0,Released,Romance under Fire!,Blockade,False,0.0,0.0,1938-06-17


  
- **se puede verificar que las filas con release_date mal ingresadas son en su conjunto datos mal ingresados en su totalidad, asi que voy a eliminarlos directamente del DF**

In [91]:
eliminar_filas = fechas_invalidas.index.to_list()
df_movies = df_movies.drop(eliminar_filas)

- **comprabamos que se hayan eliminado correctamente**

In [92]:
df_movies[19713:19716]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
19713,False,0,None,27058,tt0171227,en,Diplomatic Siege,When Steve Mitchell is clandestinely sent to t...,0.571035,/tLsQcikwkWKQiD7IHO0WIVe7m0Y.jpg,1999-11-17,0.0,90.0,Released,None,Diplomatic Siege,False,4.7,3.0,1999-11-17
19715,False,21000000,http://silverliningsplaybookmovie.com/,82693,tt1045658,en,Silver Linings Playbook,After spending eight months in a mental instit...,14.488111,/ilrZAV2klTB0FLxLb01bOp5pzD9.jpg,2012-09-08,205738714.0,122.0,Released,Watch For The Signs,Silver Linings Playbook,False,7.0,4840.0,2012-09-08
19716,False,31000000,http://www.paramount.com/flight,87502,tt1907668,en,Flight,Commercial airline pilot Whip Whitaker has a p...,10.973822,/8XEbkjgorAm9xk15mF63jqJ4ByV.jpg,2012-11-02,161772375.0,138.0,Released,None,Flight,False,6.5,2449.0,2012-11-02


In [93]:
df_movies[29471:29474]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
29473,False,0,None,50794,tt0327698,en,The Even Stevens Movie,The Stevens think that they've won an all-expe...,3.352237,/iXq9a9QDq5ZNPDsHgM6HJgT42QM.jpg,2003-06-13,0.0,120.0,Released,None,The Even Stevens Movie,False,5.6,46.0,2003-06-13
29474,False,0,None,111771,tt2006869,en,The Four Year Plan,"In 2007 Queens Park Rangers, a small Football ...",0.079713,/hlemWZPI8Ijl2TKR6HFhBFoJaSP.jpg,2011-11-16,0.0,99.0,Released,None,The Four Year Plan,False,7.0,2.0,2011-11-16
29475,False,650000,http://www.inthecrosswind.com,262009,tt2534660,et,Risttuules,"June 14, 1941, 3 a.m. Over 40000 people from E...",1.164299,/6gp1j8ezxbaIzDZFto94vLZPwan.jpg,2014-03-28,0.0,86.0,Released,None,In the Crosswind,False,7.5,14.0,2014-03-28


In [94]:
df_movies[35542:35545]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas
35545,False,0,None,163890,tt0826039,en,Blokada,The images comprise only of material Sergei Lo...,0.480285,/bXBRVGVTXxNQ6YjUZ8stYjzpifz.jpg,2005-11-07,0.0,52.0,Released,None,Blockade,False,7.0,5.0,2005-11-07
35546,False,0,None,95134,tt0012952,en,Blood and Sand,Juan is the son of a poor widow in Seville. Ag...,0.270647,/op0LsbucryeyRXE848l5QPA0miA.jpg,1922-08-05,0.0,80.0,Released,None,Blood and Sand,False,5.3,4.0,1922-08-05
35547,False,0,None,99729,tt0023878,en,Central Airport,Aviator Jim Blaine and his brother Neil are ri...,0.523341,/sqwlVQ220OHrb9TLmfixyVdhutQ.jpg,1933-04-15,0.0,72.0,Released,A Man's Courage and a Woman's Faith Put to the...,Central Airport,False,6.1,4.0,1933-04-15


---
---

- ## Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.



In [95]:
#lanza error ya que la columna budget esta como objeto
#df_movies['retorno'] = df_movies['revenue']/df_movies['budget']
df_movies['budget'] = df_movies['budget'].astype(float)
df_movies['retorno'] = df_movies['revenue']/df_movies['budget']

df_movies[['budget','revenue','retorno']]

,budget,revenue,retorno
0,30000000.0,373554033.0,12.451801
1,65000000.0,262797249.0,4.043035
2,0.0,0.0,NaN
3,16000000.0,81452156.0,5.090760
4,0.0,76578911.0,inf
...,...,...,...
45374,0.0,0.0,NaN
45375,0.0,0.0,NaN
45376,0.0,0.0,NaN
45377,0.0,0.0,NaN


In [96]:
df_movies['retorno'] = df_movies['retorno'].fillna(0)
#tambien hay valores 'np.inf' en las filas resultado de dividir un numero por 0 (cuando no esta el valor de budget)
df_movies['retorno'] = df_movies['retorno'].replace([np.inf],0)
df_movies

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,revenue,runtime,status,tagline,title,video,vote_average,vote_count,fechas_validas,retorno
0,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,373554033.0,81.0,Released,None,Toy Story,False,7.7,5415.0,1995-10-30,12.451801
1,False,65000000.0,None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995-12-15,4.043035
2,False,0.0,None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995-12-22,0.000000
3,False,16000000.0,None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995-12-22,5.090760
4,False,0.0,None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995-02-10,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45374,False,0.0,None,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,...,0.0,104.0,Released,None,Robin Hood,False,5.7,26.0,1991-05-13,0.000000
45375,False,0.0,None,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,0.0,360.0,Released,None,Century of Birthing,False,9.0,3.0,2011-11-17,0.000000
45376,False,0.0,None,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,0.0,90.0,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003-08-01,0.000000
45377,False,0.0,None,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,0.0,87.0,Released,None,Satan Triumphant,False,0.0,0.0,1917-10-21,0.000000


---
---

- ## Eliminar las columnas que no serán utilizadas para las 6 primeras consultas, video,imdb_id,adult,original_title,poster_path y homepage.

In [97]:
df_movies = df_movies.drop(["video","original_language",'runtime','overview','original_language','status','fechas_validas',"imdb_id","adult","original_title","poster_path", "homepage"], axis=1)
df_movies.head()

,budget,id,popularity,release_date,revenue,tagline,title,vote_average,vote_count,retorno
0,30000000.0,862,21.946943,1995-10-30,373554033.0,None,Toy Story,7.7,5415.0,12.451801
1,65000000.0,8844,17.015539,1995-12-15,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035
2,0.0,15602,11.712900,1995-12-22,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000
3,16000000.0,31357,3.859495,1995-12-22,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760
4,0.0,11862,8.387519,1995-02-10,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000


**Cargamos los datos de las fechas para que podamos ingresar la consulta por dia en español**

In [98]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

# Crear nuevas columnas con el día, mes y año en español
df_movies['dia'] = df_movies['release_date'].dt.day
df_movies['dia_semana'] = df_movies['release_date'].dt.day_name().apply(lambda x: {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}[x])
df_movies['mes'] = df_movies['release_date'].dt.month.apply(lambda x: ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'][x-1])
df_movies['anio'] = df_movies['release_date'].dt.year

In [99]:
df_movies = df_movies.drop('release_date', axis=1)

In [100]:
#ArrowInvalid: ("Could not convert '0.837228' with type str: tried to convert to double", 'Conversion failed for column popularity with type object')
#se ingresa esta linea para evitar el error de conversion debido a que popularity esta en formato string y hay algunos valores mal almacenados
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')
df_movies.to_parquet("../consultas/movies.parquet")

In [101]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45378
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        45376 non-null  float64
 1   id            45376 non-null  int64  
 2   popularity    45376 non-null  float64
 3   revenue       45376 non-null  float64
 4   tagline       20398 non-null  object 
 5   title         45376 non-null  object 
 6   vote_average  45376 non-null  float64
 7   vote_count    45376 non-null  float64
 8   retorno       45376 non-null  float64
 9   dia           45376 non-null  int32  
 10  dia_semana    45376 non-null  object 
 11  mes           45376 non-null  object 
 12  anio          45376 non-null  int32  
dtypes: float64(6), int32(2), int64(1), object(4)
memory usage: 4.5+ MB
